# Проект "Сервис книг"

## Введение

### Описание проекта

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.


В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Цель проекта

Изучить данные и дать рекомендации по ценностному предложению для нового продукта.

### Ход решения

1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг для книг с количеством оценок 50 и более;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Описание данных

**Таблица books**\
Содержит данные о книгах:
1. book_id — идентификатор книги;
2. author_id — идентификатор автора;
3. title — название книги;
4. num_pages — количество страниц;
5. publication_date — дата публикации книги;
6. publisher_id — идентификатор издателя.


**Таблица authors**\
Содержит данные об авторах:
1. author_id — идентификатор автора;
2. author — имя автора.


**Таблица publishers**\
Содержит данные об издательствах:
1. publisher_id — идентификатор издательства;
2. publisher — название издательства;


**Таблица ratings**\
Содержит данные о пользовательских оценках книг:
1. rating_id — идентификатор оценки;
2. book_id — идентификатор книги;
3. username — имя пользователя, оставившего оценку;
4. rating — оценка книги.


**Таблица reviews**
Содержит данные о пользовательских обзорах:
1. review_id — идентификатор обзора;
2. book_id — идентификатор книги;
3. username — имя автора обзора;
4. text — текст обзора.

## Изучение данных

In [1]:
# импортируем библиотеки
import pandas as pd 
from sqlalchemy import create_engine 
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
 db_config['pwd'], 
 db_config['host'], 
 db_config['port'], 
 db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
#Изучим датасеты
b_query = ''' SELECT * FROM books LIMIT 1 '''
a_query = ''' SELECT * FROM authors LIMIT 1 '''
p_query = ''' SELECT * FROM publishers LIMIT 1 '''
r_query = ''' SELECT * FROM ratings LIMIT 1 '''
rw_query = ''' SELECT * FROM reviews LIMIT 1 '''

In [3]:
books = pd.io.sql.read_sql(b_query, con = engine)
authors= pd.io.sql.read_sql(a_query, con = engine)
publishers = pd.io.sql.read_sql(p_query, con = engine)
ratings = pd.io.sql.read_sql(r_query, con = engine)
reviews = pd.io.sql.read_sql(rw_query, con = engine)

In [4]:
display(books)
display(authors)
display(publishers)
display(ratings)
reviews

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


,author_id,author
0,1,A.S. Byatt


,publisher_id,publisher
0,1,Ace


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


In [5]:
#Количество книг, вышедших после 1 января 2000 года;
books_after_01_01_2000_query = (
    '''
    SELECT COUNT(book_id) as count_books_after_01_01_2000
    FROM books
    WHERE publication_date > '2000-01-01'
    '''
)



In [6]:
books_after_01_01_2000 = pd.io.sql.read_sql(books_after_01_01_2000_query, con = engine)

In [7]:
books_after_01_01_2000

,count_books_after_01_01_2000
0,819


In [8]:
#Количество обзоров и средняя оценка у книг
books_avg_query = (
    '''
    WITH
    b AS (SELECT book_id, title
          FROM books),
    rw AS (SELECT book_id,
                  COUNT(review_id) as count_reviews
          FROM reviews
          GROUP BY book_id),
    r AS (SELECT book_id,
                  AVG(rating) as avg_rating
          FROM ratings
          GROUP BY book_id)
    SELECT b.book_id,
           b.title,
           rw.count_reviews,
           r.avg_rating
    FROM b
    LEFT JOIN rw ON b.book_id=rw.book_id
    LEFT JOIN r ON b.book_id=r.book_id 
    ORDER BY r.avg_rating DESC
    '''
)


In [9]:
books_avg = pd.io.sql.read_sql(books_avg_query, con = engine)

In [10]:
books_avg

,book_id,title,count_reviews,avg_rating
0,86,Arrows of the Queen (Heralds of Valdemar #1),2.0,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2.0,5.00
2,390,Light in August,2.0,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2.0,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2.0,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,316,His Excellency: George Washington,2.0,2.00
997,202,Drowning Ruth,3.0,2.00
998,371,Junky,2.0,2.00


In [11]:
#Издательство, которое выпустило наибольшее число книг толще 50 страниц 
best_publish_query = (
    '''
    SELECT publisher,
           b.publisher_id,
           COUNT(b.book_id) AS count_books
    FROM books b
    LEFT JOIN publishers AS p ON p.publisher_id=b.publisher_id
    WHERE num_pages>50
    GROUP BY b.publisher_id,
             publisher  
    ORDER BY count_books DESC
    LIMIT 1
    '''
)

In [12]:
best_publish = pd.io.sql.read_sql(best_publish_query, con = engine)

In [13]:
best_publish

,publisher,publisher_id,count_books
0,Penguin Books,212,42


Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [67]:
#Автор с самой высокой средней оценкой книг
#для книг с количеством оценок 50 и более
top_author_query = (
    '''
    SELECT a.author,
           AVG(rating) as avg_rating
    FROM books b
    LEFT JOIN  ratings r ON b.book_id=r.book_id 
    LEFT JOIN authors a ON a.author_id=b.author_id
    WHERE b.book_id IN (SELECT book_id
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating_id)>49                        
                       )
    GROUP BY a.author_id
    ORDER BY avg_rating DESC
    LIMIT 1
    '''
)

In [68]:
top_author = pd.io.sql.read_sql(top_author_query, con = engine)

In [69]:
top_author

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


In [ ]:
#Среднее количество обзоров от пользователей,
#которые поставаили больше 50 оценок
avg_review_query = (
    '''
    WITH
    r AS (SELECT username,
                 COUNT(review_id)
          FROM reviews rw
          WHERE username IN (SELECT username
                           FROM ratings
                           GROUP BY username
                           HAVING COUNT(rating_id)>50
                           )
          GROUP BY username)
    SELECT AVG(count) AS avg_review_users_more_50_ratings
    FROM r
    '''
)

In [ ]:
avg_review = pd.io.sql.read_sql(avg_review_query, con = engine)

In [ ]:
avg_review

## Вывод
В данном проекте мы подключились к базе данных и выгрузили датасеты, используя SQL. В нашем распоряжении 5 таблиц и схема данных с указанием первичного и внешнего ключей. 

Первым шагом мы изучили, сколько у нас на сервисе книг из 21 века. Их количество составило 819 ед.
Затем мы выгрузили книги с указанием количества обзоров и средних оценок, отсортировав по среднему рейтингу по убыванию. У лидеров средний рейтинг составляет 5.0, количество обзоров 2.0. Это говорит о том, что у книги очень мало оценок, поэтому пока доверять данным критериям не стоит.
Лидером по наибольшему количеству, выпущенных книг, стал Penguin Books. Для оценки критерия «книга» мы поставили условие, что к «книге» мы относим издание свыше 50 страниц, с целью отсеять брошюры.
При определении лучшего автора по средней оценке, мы учитывали количество оценок. Их должно было быть больше 50. При учете условия в количестве оценок, лидером стала Joanne Rowling с наибольшим средним рейтингом в 4.288.
Наиболее активные пользователи, которые поставили свыше 50 оценок, периодически пишут обзоры. Среднее количество обзоров от таких пользователей составляет 24.3.

Учитывая, полученные выше итоги, можно предложить следующую стратегию развития:\
•	Пополнение сервиса книгами 21 века.\
•	Установить на странице книге не только рейтинг книги и количество обзоров, но и количество пользователей, оценивших книгу, а также количество пользователей, прочитавших книгу. Это поможет и пользователям, и владельцам сервиса понимать наиболее интересные/востребованные книги.\
•	Определенно следует взаимодействовать с издателями такими, как Penguin Books. Так как у таких издателей большое количество, выпускаемых книг. Это позволит увеличить библиотеку сервиса.\
•	На главной странице сайта/приложения следует расположить раздел с авторами, средний рейтинг которых выше 4 (при количестве оценок больше 50). Скорее всего книги этих авторов являются бестселлерами.\ 
•	Взаимодействие пользователей с приложением сильно помогает понимание клиента, как следствие развитие бизнеса. Можно продумать систему подарков или скидок для пользователей написавших больше 20 обзоров и поставивших больше 50 оценок в год.